In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")
#Conectamos Google Drive en el entorno de Google Colab para poder acceder a los archivos almacenados en el Drive y utilizarlos en el  Colab.

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import sqlite3 #Proporciona una interfaz para trabajar con bases de datos SQLite en tus programas de Python.

In [ ]:
ls -la # listar archivos y directorios en el sistema.

total 20
drwxr-xr-x 1 root root 4096 Oct  6 20:36 ./
drwxr-xr-x 1 root root 4096 Oct  6 20:36 ../
drwxr-xr-x 4 root root 4096 Oct  5 19:32 .config/
drwx------ 6 root root 4096 Oct  6 20:36 gdrive/
drwxr-xr-x 1 root root 4096 Oct  5 19:33 sample_data/


#**Creamos la base de datos**

In [ ]:
import sqlite3 #Importa el módulo sqlite3, que proporciona una interfaz para trabajar con bases de datos SQLite desde Python.


conn= sqlite3.connect('TestDB_A.db') #Crea una conexión a la base de datos SQLite llamada 'TestDB_A.db'. Si la base de datos no existe, se creará automáticamente en la ubicación donde se ejecuta el código.

c = conn.cursor() #Se crea un cursor

# Creamos una tabla llamada - GN
c.execute('''CREATE TABLE GN
             ([ISO3] Text PRIMARY KEY,[Country] text, [hdicode] text, [gnipc_2017] float, [gnipc_2018] float, [gnipc_2019] float, [gnipc_2020] float, [gnipc_2021] float)''')

# Creamos una tabla llamada -HD
c.execute('''CREATE TABLE HD
              ([ISO3] Text PRIMARY KEY,[Country] text, [hdicode] text, [hdi_2017] float, [hdi_2018] float, [hdi_2019] float, [hdic_2020] float, [hdi_2021] float)''')

# Creamos una tabla llamada - LE
c.execute('''CREATE TABLE LE
             ([ISO3] Text PRIMARY KEY,[Country] text, [hdicode] text, [le_2017] float, [le_2018] float, [le_2019] float, [le_2020] float, [le_2021] float)''')
# Creamos una tabla llamada NEW_TABLE
c.execute('''CREATE TABLE NEW_TABLE
             ([Country] text, [hdi_2021] float,[le_2021] float)''')

conn.commit()
#Se usa para confirmar los cambios en la base de datos.
#Esto es necesario para que las tablas se creen realmente en la base de datos.


In [ ]:
ls -la

total 52
drwxr-xr-x 1 root root  4096 Oct  6 20:37 ./
drwxr-xr-x 1 root root  4096 Oct  6 20:36 ../
drwxr-xr-x 4 root root  4096 Oct  5 19:32 .config/
drwx------ 6 root root  4096 Oct  6 20:36 gdrive/
drwxr-xr-x 1 root root  4096 Oct  5 19:33 sample_data/
-rw-r--r-- 1 root root 32768 Oct  6 20:37 TestDB_A.db


In [ ]:
#%pwd

#Importamos datos


In [ ]:
import sqlite3
import pandas as pd
from pandas import DataFrame

conn = sqlite3.connect('TestDB_A.db') #Crea una conexión a la base de datos SQLite llamada 'TestDB_A.db'
c = conn.cursor() #Creamos otro cursor
read_HD = pd.read_excel(r'/content/gdrive/My Drive/T_S_C/hdi.xlsx')
read_HD.to_sql('HD', conn, if_exists='replace', index = False)

read_GN = pd.read_excel (r'/content/gdrive/My Drive/T_S_C/NG.xlsx')
read_GN.to_sql('NG', conn, if_exists='replace', index = False)

read_LE = pd.read_excel(r'/content/gdrive/My Drive/T_S_C/le.xlsx')
read_LE.to_sql('LE', conn, if_exists='replace', index = False)
#Lee tres archivos Excel diferentes ('hdi.xlsx', 'NG.xlsx' y 'le.xlsx') que se encuentran en la ubicación '/content/gdrive/My Drive/T_S_C/'.

195

In [ ]:
# Realiza la inserción
c.execute('''
INSERT INTO NEW_TABLE (Country, hdi_2021, le_2021)
SELECT DISTINCT h.Country, h.hdi_2021, l.le_2021
FROM HD AS h
LEFT JOIN NG AS n ON h.Country = n.Country
LEFT JOIN LE AS l ON h.Country = l.Country
WHERE l.le_2021 > 70
''')

# Realiza una consulta de selección para obtener los datos insertados
c.execute('''
SELECT *
FROM NEW_TABLE
''')

# Obtén los resultados en un DataFrame
df = DataFrame(c.fetchall(), columns=["Country", "hdi_2021", "le_2021"])
print(df)


                                Country  hdi_2021  le_2021
0                               Albania     0.796  76.4626
1                               Andorra     0.858  80.3684
2                  United Arab Emirates     0.911  78.7104
3                             Argentina     0.842  75.3899
4                               Armenia     0.759  72.0431
..                                  ...       ...      ...
115                          Uzbekistan     0.727  70.8616
116  Venezuela (Bolivarian Republic of)     0.691  70.5536
117                            Viet Nam     0.703  73.6181
118                             Vanuatu     0.607  70.4490
119                               Samoa     0.707  72.7675

[120 rows x 3 columns]


In [ ]:
# Ejecuta una consulta SQL para encontrar el país, hdi_2021 y le_2021 correspondientes al número más grande en hdi_2021
c.execute('''
SELECT Country, hdi_2021, le_2021
FROM NEW_TABLE
WHERE hdi_2021 = (SELECT MAX(hdi_2021) FROM NEW_TABLE)
''')

# Recupera el resultado de la consulta en un DataFrame
df_max_hdi_info = DataFrame(c.fetchall(), columns=["Country", "hdi_2021", "le_2021"])

# Muestra el país, hdi_2021 y le_2021 correspondientes al número más grande en hdi_2021
print("País, hdi_2021 y le_2021 correspondientes al número más grande en hdi_2021:")
print(df_max_hdi_info)


País, hdi_2021 y le_2021 correspondientes al número más grande en hdi_2021:
       Country  hdi_2021  le_2021
0  Switzerland     0.962  83.9872


In [ ]:
# Calcula el promedio de las columnas "hdi_2021" y "le_2021"
promedio_hdi = df["hdi_2021"].mean()
promedio_le = df["le_2021"].mean()

# Muestra el promedio
print("Promedio de hdi_2021:", promedio_hdi)
print("Promedio de le_2021:", promedio_le)



Promedio de hdi_2021: 0.8092627118644068
Promedio de le_2021: 76.26118916666668


In [ ]:
# Ejecuta la consulta SQL para encontrar los registros donde hdi_2021 es igual a 0.478, eliminando duplicados
c.execute('''
SELECT DISTINCT Country, hdi_2021, le_2021
FROM NEW_TABLE
WHERE hdi_2021 = 0.478
''')

# Recupera el resultado de la consulta en un DataFrame
df_hdi_478 = DataFrame(c.fetchall(), columns=["Country", "hdi_2021", "le_2021"])

# Muestra los registros donde hdi_2021 es igual a 0.478 sin duplicados
print("Registros donde hdi_2021 es igual a 0.478 sin duplicados:")
print(df_hdi_478)


Registros donde hdi_2021 es igual a 0.478 sin duplicados:
Empty DataFrame
Columns: [Country, hdi_2021, le_2021]
Index: []


In [ ]:
# Ejecuta la consulta SQL para encontrar los registros donde hdi_2021 es mayor que 0.900
c.execute('''
SELECT DISTINCT Country, hdi_2021, le_2021
FROM NEW_TABLE
WHERE hdi_2021 > 0.900
''')

# Recupera el resultado de la consulta en un DataFrame
df_hdi_900 = DataFrame(c.fetchall(), columns=["Country", "hdi_2021", "le_2021"])

# Muestra los registros donde hdi_2021 es mayor que 0.900
print("Registros donde hdi_2021 es mayor que 0.900:")
print(df_hdi_900)


Registros donde hdi_2021 es mayor que 0.900:
                   Country  hdi_2021  le_2021
0     United Arab Emirates     0.911  78.7104
1                Australia     0.951  84.5265
2                  Austria     0.916  81.5797
3                  Belgium     0.937  81.8787
4                   Canada     0.936  82.6565
5              Switzerland     0.962  83.9872
6                  Germany     0.942  80.6301
7                  Denmark     0.948  81.3753
8                    Spain     0.905  83.0100
9                  Finland     0.940  82.0381
10                  France     0.903  82.4988
11          United Kingdom     0.929  80.7422
12  Hong Kong, China (SAR)     0.952  85.4734
13                 Ireland     0.945  81.9976
14                 Iceland     0.959  82.6782
15                  Israel     0.919  82.2550
16                   Japan     0.925  84.7839
17     Korea (Republic of)     0.925  83.6978
18           Liechtenstein     0.935  83.2575
19              Luxembourg     0.93

In [ ]:
# Ejecuta la consulta SQL para encontrar los registros que cumplan ambas condiciones
c.execute('''
SELECT DISTINCT Country, hdi_2021, le_2021
FROM NEW_TABLE
WHERE hdi_2021 > 0.900 AND le_2021 < 80
''')

# Recupera el resultado de la consulta en un DataFrame
df_hdi80 = DataFrame(c.fetchall(), columns=["Country", "hdi_2021", "le_2021"])

# Muestra los registros que cumplen ambas condiciones
print("Registros donde hdi_2021 es mayor que 0.900 y le_2021 es menor que 80:")
print(df_hdi80)


Registros donde hdi_2021 es mayor que 0.900 y le_2021 es menor que 80:
                Country  hdi_2021  le_2021
0  United Arab Emirates     0.911  78.7104
1         United States     0.921  77.1982


# **PARTE 2**

/*Importamos datos desde archivos de excel*/
FILENAME REFFILE '/home/u63628016/Tarea2/hdi.xlsx';

PROC IMPORT DATAFILE=REFFILE
	DBMS=XLSX
	OUT=WORK.IMPORT1;
	GETNAMES=YES;
RUN;

FILENAME REFFILE '/home/u63628016/Tarea2/le.xlsx';

PROC IMPORT DATAFILE=REFFILE
	DBMS=XLSX
	OUT=WORK.IMPORT2;
	GETNAMES=YES;
RUN;

FILENAME REFFILE '/home/u63628016/Tarea2/NG.xlsx';

PROC IMPORT DATAFILE=REFFILE
	DBMS=XLSX
	OUT=WORK.IMPORT3;
	GETNAMES=YES;
RUN;

/*Unimos tablas para crear una nueva*/
PROC SQL;
   CREATE TABLE work.combined_data AS
   SELECT *
   FROM WORK.IMPORT1
   FULL JOIN WORK.IMPORT2
   ON IMPORT1.Country = IMPORT2.Country;
QUIT;

/*Para encontrar el país, hdi_2021 y le_2021 correspondientes al máximo hdi_2021*/
PROC SQL;
SELECT Country, hdi_2021, le_2021
FROM work.combined_data
WHERE hdi_2021 = (SELECT MAX(hdi_2021) FROM work.combined_data);
QUIT;

/*Para encontrar los registros donde hdi_2021 es igual a 0.478, eliminando duplicados*/
PROC SQL;
SELECT DISTINCT Country, hdi_2021, le_2021
FROM work.combined_data
WHERE hdi_2021 = 0.478;
QUIT;

/*Para encontrar los registros donde hdi_2021 es mayor que 0.900*/
PROC SQL;
SELECT DISTINCT Country, hdi_2021, le_2021
FROM work.combined_data
WHERE hdi_2021 > 0.900;
QUIT;

/*Para encontrar los registros donde hdi_2021 es mayor que 0.900 y le_2021 es menor que 80*/
PROC SQL;
SELECT DISTINCT Country, hdi_2021, le_2021
FROM work.combined_data
WHERE hdi_2021 > 0.900 AND le_2021 < 80;
QUIT;

/*vamos a calcular la media*/
PROC MEANS DATA=work.combined_data MEAN;
VAR hdi_2021 le_2021;
RUN;

# **CONCLUCIONES**

Se puede observar que ambos programas dan resultados en una gran cantidad de datos lo cual ayuda a recopilar datos de manera casi instantanea, tanto phyton oscomo SAS dan los mismos resultados a las busquedas que hicimos, diferentes terminos sin embargo mismo resultado lo cual ocuparemos en el futuro.